# Práctica 6.  Modelo Simple de Combustión

**Nombre:** Heriberto Espino Montelongo

**Materia:** Análisis numérico

**Sección:** 1

**Fecha:** 11/11/2024

# Instrucciones

Considerando el problema de valor inicial (PVI):

Resolver:
$$
\frac{dy}{dt} = f(t, y) \tag{1}
$$

Sujeta a:
$$
y(t_0) = y_0 \tag{2}
$$

# Funciones

In [1]:
import matplotlib.pyplot as plt
import mpld3
from IPython.display import display, HTML

import numpy as np
import sympy as sp
from sympy.abc import x 

from matplotlib.backends.backend_pdf import PdfPages

from prettytable import PrettyTable

In [2]:
all_figures = []

In [3]:
def plot_solution(t, y, fig_title, color):

    global all_figures  # Use global to store figures across multiple calls

    fig = plt.figure(figsize=(8, 6), dpi=150) 
    plt.plot(t, y,'o', alpha=0.9, color=color)
    plt.title(fig_title, fontsize=14, fontweight='bold')
    plt.grid()
    plt.xlabel('t')
    plt.ylabel('y')

    display(HTML(mpld3.fig_to_html(plt.gcf()))) # Display the plot as HTML
    plt.close() # Close the plot to prevent it from displaying twice

    all_figures.append(fig) # Store the figure to save it later
    
    return

In [4]:
def exact_solution(name, function, lower_bound, upper_bound, h, plot = False):

    f = sp.lambdify(x,function)
    n = int((upper_bound-lower_bound)/h)
    t = np.linspace(lower_bound,upper_bound,n+1)
    y = f(t)

    if plot:
        plot_solution(t, y, 'Heri - Exact Solution - ' + str(name), '#B65A5A')

    return t, y

In [5]:
def euler(name, f, t0, tn, h, y0, plot = False):

    n = int(abs(tn - t0) / h)
    t = np.linspace(t0, tn, n + 1)
    y = np.zeros(n + 1)
    y[0] = y0
    for k in range(n):
        y[k + 1] = y[k] + h * f(t[k], y[k])

    if plot:
        plot_solution(t, y, 'Heri - Euler\'s method - ' + str(name), '#1A5465')

    return t, y

In [6]:
def RK4(name, f, t0, tn, h, y0, plot = False):

    n = int(abs(tn - t0) / h)
    t = np.linspace(t0, tn, n + 1)
    y = np.zeros(n + 1)
    y[0] = y0
    
    for i in range(n):
        s1 = f(t[i], y[i])
        s2 = f(t[i] + h / 2, y[i] + s1 * h / 2)
        s3 = f(t[i] + h / 2, y[i] + s2 * h / 2)
        s4 = f(t[i] + h, y[i] + s3 * h)
        y[i + 1] = y[i] + h * (s1 + 2 * s2 + 2 * s3 + s4) / 6

    if plot:
        plot_solution(t, y, 'Heri - RK4 method - ' + str(name), '#1A5465')

    return t, y

In [7]:
def compare_methods(name, function, f, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, function, t0, tn, h)
    t_euler, y_euler = euler(name, f, t0, tn, h, y0)
    t_rk4, y_rk4 = RK4(name, f, t0, tn, h, y0)

    global all_figures

    fig = plt.figure(figsize=(5, 3), dpi=150) 
    plt.plot(t_exact, y_exact, '-', label='Exact Solution', color='#50A3A4', alpha=0.5, lwd=2)
    plt.plot(t_euler, y_euler, 's--', label='Euler\'s Method', color='#FCAF38', alpha=0.5)
    plt.plot(t_rk4, y_rk4, 'o--', label='RK4 Method', color='#F95335', alpha=0.5)
    plt.title('Heri - Comparison of Methods - ' + str(name), fontsize=14, fontweight='bold')
    plt.xlabel('t')
    plt.ylabel('y')
    plt.legend()
    plt.grid()

    display(HTML(mpld3.fig_to_html(plt.gcf())))
    plt.close()

    all_figures.append(fig)
    
    return  

In [8]:
def euler_vs_RK4(name, f, t0, tn, h, y0, plot = False, table = False, final_results = False):

    t_euler, y_euler = euler(name, f, t0, tn, h, y0)
    t_rk4, y_rk4 = RK4(name, f, t0, tn, h, y0)

    if table:
        myTable = PrettyTable(["x_i", "y_i Euler", "y_i RK4"])
        myTable.title = "Euler vs RK4 " + str(name)

        for i in range(len(t_euler)):
            myTable.add_row([
                round(t_euler[i], 2), 
                round(y_euler[i], 2), 
                round(y_rk4[i], 2), 
            ])

        display(HTML(myTable.get_html_string()))

    if plot:
        global all_figures

        fig = plt.figure(figsize=(5, 3), dpi=150) 
        plt.plot(t_euler, y_euler, 's-', label="Euler's Method", color='#1A5465', alpha=0.5)
        plt.plot(t_rk4, y_rk4, 'd-', label='RK4 Method', color='#2A9D8F', alpha=0.5)
        plt.title('Heri - Comparison of Methods - ' + str(name), fontsize=14, fontweight='bold')
        plt.xlabel('t')
        plt.ylabel('y')
        plt.legend()
        plt.grid()
        
        display(HTML(mpld3.fig_to_html(plt.gcf())))
        plt.close()

        all_figures.append(fig)

    if final_results:
        print(f'{name}\nEuler\'s Method: {y_euler[-1]}\nRK4 Method: {y_rk4[-1]}\n')

    return

In [9]:
def euler_vs_RK4_table(name, f, t0, tn, h, y0):

    t_euler, y_euler = euler(name, f, t0, tn, h, y0)
    t_rk4, y_rk4 = RK4(name, f, t0, tn, h, y0)
    
    myTable = PrettyTable(["x_i", "y_i Euler", "y_i RK4"])
    myTable.title = "Euler vs RK4 " + str(name)

    for i in range(len(t_euler)):
        myTable.add_row([
            round(t_euler[i], 2), 
            round(y_euler[i], 2), 
            round(y_rk4[i], 2), 
        ])

    display(HTML(myTable.get_html_string()))

In [10]:
def error_euler(name, f, function, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, f, t0, tn, h)
    t_euler, y_euler = euler(name, function, t0, tn, h, y0)
    
    myTable = PrettyTable(["x_i", "y_i Exact", "y_i Euler", "Absolute E", "Relative E"])
    myTable.title = "Error Analysis for Euler  " + str(name)

    for i in range(len(t_exact)):
        myTable.add_row([
            round(t_exact[i], 2), 
            round(y_exact[i], 2), 
            round(y_euler[i], 2), 
            round(abs(y_exact[i] - y_euler[i]), 4), 
            round(abs(y_exact[i] - y_euler[i]) / y_exact[i], 4)
        ])

    display(HTML(myTable.get_html_string()))

In [11]:
def error_rk4(name, f, function, t0, tn, h, y0):

    t_exact, y_exact = exact_solution(name, f, t0, tn, h)
    t_rk4, y_rk4 = RK4(name, function, t0, tn, h, y0)

    myTable = PrettyTable(["x_i", "y_i Exact", "y_i RK4", "Absolute E", "Relative E"])
    myTable.title = "Error Analysis for RK4  " + str(name)

    for i in range(len(t_exact)):
        myTable.add_row([
            round(t_exact[i], 2), 
            round(y_exact[i], 2), 
            round(y_rk4[i], 2), 
            round(abs(y_exact[i] - y_rk4[i]), 4), 
            round(abs(y_exact[i] - y_rk4[i]) / y_rk4[i], 4)
        ])
        
    display(HTML(myTable.get_html_string()))

In [12]:
def multipage(filename, figs=None, dpi=150):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]  # Fallback: get all open figures
    for fig in figs:
        # Save each figure with tight layout and specified dpi to control size
        fig.savefig(pp, format='pdf', dpi=dpi, bbox_inches='tight')
    pp.close()

# Problema

Considera el modelo simple de combustión dado por
   $$
   y' = y^2 - y^3, \quad y(0) = y_0, \quad 0 \leq t \leq \frac{2}{y_0}
   $$
   donde $y$ es el radio de la bola de llama. Utiliza RK4 y el método de Euler para investigar el modelo de la llama con estos parámetros: $y_0 = 0.01$ y $h = \frac{t_{\text{final}}}{500}$. Grafica la solución numérica con ambos métodos numéricos. Superpone ambas curvas de solución en el mismo sistema de coordenadas. Si es posible, utiliza un color diferente para cada curva. Interpreta la solución.

In [13]:
def combustion_model(t, y):
    return y**2 - y**3

t0 = 0 
y0 = 0.01
tn = 2/y0
h = tn/500

name = f"Modelo simple de combustion, h = {h}"

euler_vs_RK4(name, combustion_model, t0, tn, h, y0, plot = True, table = True, final_results = False)

x_i,y_i Euler,y_i RK4
0.0,0.01,0.01
0.4,0.01,0.01
0.8,0.01,0.01
1.2,0.01,0.01
1.6,0.01,0.01
2.0,0.01,0.01
2.4,0.01,0.01
2.8,0.01,0.01
3.2,0.01,0.01
3.6,0.01,0.01


La gráfica muestra el crecimiento del radio $y$ de la llama, donde ambas curvas presentan un patrón de crecimiento rápido entre los valores de tiempo alrededor de $t \approx 100$ y $t \approx 110$. En este intervalo, el radio pasa de valores pequeños a alcanzar su máximo en $y = 1$, lo que indica un cambio abrupto en el proceso de combustión.


# Exportar los plots

In [14]:
multipage("Práctica 6 Modelo Simple de Combustión.pdf", figs=all_figures)